In [3]:
%load_ext autoreload
%autoreload 2

In [84]:
import numpy as np

from models import Sequential
from layers import Dense
from losses import CategoricalCrossentropy, BinaryCrossentropy
from activations import Softmax, Sigmoid

from sklearn.metrics import log_loss

loss1 = CategoricalCrossentropy()
loss2 = BinaryCrossentropy()

y_true = np.array([[1, 0],
                    [0, 1]])

z = np.array([[4, -4],
              [-6, 6]])

print(Sigmoid()(z[:,0,None]))
print(Softmax()(z))
print()
print(Sigmoid().derivative(z[:,0,None]))
print(Softmax().derivative(z))

# print(loss1(y_true, y_pred))
# print(loss2(y_true[:,0,None], y_pred[:,0,None]))
# print(log_loss(y_true, y_pred))
# print()
# print(loss1.derivative(y_true, y_pred)/2)
# print(loss2.derivative(y_true[:,0,None], y_pred[:,0,None])/2)
# print()
# print(y_pred + loss1.derivative(y_true, y_pred)/2)
# print(y_pred[:,0,None] + loss2.derivative(y_true[:,0,None], y_pred[:,0,None])/2)

[[0.98201379]
 [0.00247262]]
[[9.99664650e-01 3.35350130e-04]
 [6.14417460e-06 9.99993856e-01]]

[[0.01766271]
 [0.00246651]]
[[3.35237671e-04 3.35237671e-04]
 [6.14413685e-06 6.14413685e-06]]
